# An Example of using `minimod`

For this notebook, we will first create the data needs to be used for the program. Then we will instantiate the class and make a report of the results using the `report` method

In [1]:
%load_ext autoreload

In [13]:
%autoreload 2
import sys
import pandas as pd
import numpy as np
import minimod as mm


In [3]:
## Data

# This is how the data was processed, assuming we use the Katie_VA_Benefits_and_Costs_1_8_2019.xlsx file.

## First we get a path to the raw data
excel_file = "data/raw/Katie_VA_Benefits_and_Costs_1_8_2019.xlsx"


# Get "vasoilold" and get discounted benefits for constraint

discount_costs = 1/(1 + 0.03)


vasoilold_constraint = (
    pd.read_excel(excel_file,
                  sheet_name = 'Benefits',
                  header = 2)
    .loc[lambda df : df['intervention'] == 'vasoilold']
    .set_index(['intervention', 'space'])
    .stack()
    .to_frame()
    .reset_index()
    .rename({
        'level_2' : 'time',
        0 : 'benefit'
    }, axis= 'columns')
    .assign(
        time_rank = lambda df: (df['time'].rank(numeric_only=True, method= 'dense') -1).astype(int) ,
        time_discount_costs = lambda df: discount_costs**df['time_rank'],
        discounted_benefits = lambda df: df['time_discount_costs']*df['benefit']
    )
    ['discounted_benefits'].sum()
    
)


In [4]:
df_benefit = (pd.read_excel(excel_file, 
                           sheet_name = 'Benefits',
                           header = 2)
              .loc[lambda df: df['intervention'] != 'vasoilold']
              .set_index(['intervention', 'space'])
              .stack()
              .to_frame()
              .reset_index()
              .rename({'level_2' : 'time',
                       0 : 'benefit'}, axis=1)
              .set_index(['intervention', 'space', 'time'])
              )

df_cost = (pd.read_excel(excel_file, 
                           sheet_name = 'Costs',
                           header = 2)
           .loc[lambda df: df['intervention'] != 'vasoilold']
              .set_index(['intervention', 'space'])
              .stack()
              .to_frame()
              .reset_index()
              .rename({'level_2' : 'time',
                       0 : 'costs'}, axis=1)
              .set_index(['intervention', 'space', 'time'])
              )

In [5]:
# Then we merge the cost and benefit data together.

df = (df_benefit
      .merge(df_cost, left_index=True, right_index=True)
      .reset_index()    
)


# Then we save the data. The finished data can be found in the `/examples/data/raw` folder.

df.to_csv("data/processed/example1.csv")


In [3]:
df = pd.read_csv("data/processed/example1.csv")

## Running the model

Now we instantiate the model, and then run `fit` and get the report.

In [14]:
c = mm.CostSolver(minimum_benefit = vasoilold_constraint)


              MiniMod Nutrition Intervention Tool
              Optimization Method: MIN
              Version: 0.0.2dev
              Solver: CBC
              
              


In [15]:
opt = c.fit(data = df, 
all_space = ['cube', 'oil', 'maize'], 
all_time = ['maize', 'cube'])

[Note]: Processing Data...


TypeError: descriptor 'values' for 'Block' objects doesn't apply to a 'FloatBlock' object

In [12]:
c.report()

opt_df = c.opt_df.reset_index()


n3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/lib/python3.8/site-packages/pandas/core/generic.py", line 151, in <module>
    class NDFrame(PandasObject, SelectionMixin, indexing.IndexingMixin):
  File "/usr/lib/python3.8/site-packages/pandas/core/generic.py", line 1111, in NDFrame
    def rename_axis(self, mapper=lib.no_default, **kwargs):
AttributeError: module 'pandas._libs.lib' has no attribute 'no_default'
]
[autoreload of pandas.plotting._matplotlib.core failed: Traceback (most recent call last):
  File "/

AttributeError: 'DataFrame' object has no attribute '_take_with_is_copy'

In [15]:
c.write()